In [1]:
import pandas as pd
import numpy as np
import json

# Loading CSV and JSON files

In [2]:
transactions_data = pd.read_csv("transactions_data.csv")
cards_data = pd.read_csv("cards_data.csv")
users_data = pd.read_csv("users_data.csv")

In [3]:
with open("mcc_codes.json", "r") as f:
    mcc_codes = json.load(f)
with open("train_fraud_labels.json", "r") as f:
    fraud_labels = json.load(f)

In [4]:
fraud_labels_df = pd.DataFrame(fraud_labels['target'].items(), columns=['id', 'label'])
fraud_labels_df.head()

,id,label
0,10649266,No
1,23410063,No
2,9316588,No
3,12478022,No
4,9558530,No


In [5]:
mcc_codes_df = pd.DataFrame(mcc_codes.items(), columns=['mcc', 'data'])
mcc_codes_df.head()

,mcc,data
0,5812,Eating Places and Restaurants
1,5541,Service Stations
2,7996,"Amusement Parks, Carnivals, Circuses"
3,5411,"Grocery Stores, Supermarkets"
4,4784,Tolls and Bridge Fees


## Removing and Renaming columns for merging Data frames

In [6]:
transactions_data.isnull().sum()
transactions_data = transactions_data.drop(columns=['merchant_state', 'zip', 'errors'])
transactions_data.head()

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,mcc
0,7475327,2010-01-01 00:01:00,1556,2972,$-77.00,Swipe Transaction,59935,Beulah,5499
1,7475328,2010-01-01 00:02:00,561,4575,$14.57,Swipe Transaction,67570,Bettendorf,5311
2,7475329,2010-01-01 00:02:00,1129,102,$80.00,Swipe Transaction,27092,Vista,4829
3,7475331,2010-01-01 00:05:00,430,2860,$200.00,Swipe Transaction,27092,Crown Point,4829
4,7475332,2010-01-01 00:06:00,848,3915,$46.41,Swipe Transaction,13051,Harwood,5813


In [7]:
cards_data.isnull().sum()
cards_data = cards_data.rename(columns = {'id' : 'card_id'})
cards_data.head()

,card_id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
0,4524,825,Visa,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No
1,2731,825,Visa,Debit,4956965974959986,12/2020,393,YES,2,$21968,04/2014,2014,No
2,3701,825,Visa,Debit,4582313478255491,02/2024,719,YES,2,$46414,07/2003,2004,No
3,42,825,Visa,Credit,4879494103069057,08/2024,693,NO,1,$12400,01/2003,2012,No
4,4659,825,Mastercard,Debit (Prepaid),5722874738736011,03/2009,75,YES,1,$28,09/2008,2009,No


In [8]:
users_data.isnull().sum()
users_data = users_data.rename(columns = {'id' : 'client_id'})
users_data.head()

,client_id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards
0,825,53,66,1966,11,Female,462 Rose Lane,34.15,-117.76,$29278,$59696,$127613,787,5
1,1746,53,68,1966,12,Female,3606 Federal Boulevard,40.76,-73.74,$37891,$77254,$191349,701,5
2,1718,81,67,1938,11,Female,766 Third Drive,34.02,-117.89,$22681,$33483,$196,698,5
3,708,63,63,1957,1,Female,3 Madison Street,40.71,-73.99,$163145,$249925,$202328,722,4
4,1164,43,70,1976,9,Male,9620 Valley Stream Drive,37.76,-122.44,$53797,$109687,$183855,675,1


## Merging CSV and JSON files

In [9]:
merged_data = pd.merge(transactions_data, cards_data, on=['card_id', 'client_id'], how='left')
merged_data.head()

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,mcc,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
0,7475327,2010-01-01 00:01:00,1556,2972,$-77.00,Swipe Transaction,59935,Beulah,5499,Mastercard,Debit (Prepaid),5497590243197280,07/2022,306,YES,2,$55,05/2008,2008,No
1,7475328,2010-01-01 00:02:00,561,4575,$14.57,Swipe Transaction,67570,Bettendorf,5311,Mastercard,Credit,5175842699412235,12/2024,438,YES,1,$9100,09/2005,2015,No
2,7475329,2010-01-01 00:02:00,1129,102,$80.00,Swipe Transaction,27092,Vista,4829,Mastercard,Debit,5874992802287595,05/2020,256,YES,1,$14802,01/2006,2008,No
3,7475331,2010-01-01 00:05:00,430,2860,$200.00,Swipe Transaction,27092,Crown Point,4829,Mastercard,Debit,5346827663529174,10/2024,54,NO,2,$37634,05/2004,2006,No
4,7475332,2010-01-01 00:06:00,848,3915,$46.41,Swipe Transaction,13051,Harwood,5813,Visa,Debit,4354185735186651,01/2020,120,YES,1,$19113,07/2009,2014,No


In [10]:
merged_data = pd.merge(merged_data, users_data, on=['client_id'], how='left')
merged_data.head()

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,mcc,card_brand,...,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards
0,7475327,2010-01-01 00:01:00,1556,2972,$-77.00,Swipe Transaction,59935,Beulah,5499,Mastercard,...,7,Female,594 Mountain View Street,46.80,-100.76,$23679,$48277,$110153,740,4
1,7475328,2010-01-01 00:02:00,561,4575,$14.57,Swipe Transaction,67570,Bettendorf,5311,Mastercard,...,6,Male,604 Pine Street,40.80,-91.12,$18076,$36853,$112139,834,5
2,7475329,2010-01-01 00:02:00,1129,102,$80.00,Swipe Transaction,27092,Vista,4829,Mastercard,...,4,Male,2379 Forest Lane,33.18,-117.29,$16894,$34449,$36540,686,3
3,7475331,2010-01-01 00:05:00,430,2860,$200.00,Swipe Transaction,27092,Crown Point,4829,Mastercard,...,5,Female,903 Hill Boulevard,41.42,-87.35,$26168,$53350,$128676,685,5
4,7475332,2010-01-01 00:06:00,848,3915,$46.41,Swipe Transaction,13051,Harwood,5813,Visa,...,5,Male,166 River Drive,38.86,-76.60,$33529,$68362,$96182,711,2


In [11]:
mcc_codes_df['mcc'] = mcc_codes_df['mcc'].astype(int)
fraud_labels_df['id'] = fraud_labels_df['id'].astype(int)

In [12]:
merged_data = pd.merge(merged_data, mcc_codes_df, on = 'mcc', how = 'left')
merged_data.head()

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,mcc,card_brand,...,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards,data
0,7475327,2010-01-01 00:01:00,1556,2972,$-77.00,Swipe Transaction,59935,Beulah,5499,Mastercard,...,Female,594 Mountain View Street,46.80,-100.76,$23679,$48277,$110153,740,4,Miscellaneous Food Stores
1,7475328,2010-01-01 00:02:00,561,4575,$14.57,Swipe Transaction,67570,Bettendorf,5311,Mastercard,...,Male,604 Pine Street,40.80,-91.12,$18076,$36853,$112139,834,5,Department Stores
2,7475329,2010-01-01 00:02:00,1129,102,$80.00,Swipe Transaction,27092,Vista,4829,Mastercard,...,Male,2379 Forest Lane,33.18,-117.29,$16894,$34449,$36540,686,3,Money Transfer
3,7475331,2010-01-01 00:05:00,430,2860,$200.00,Swipe Transaction,27092,Crown Point,4829,Mastercard,...,Female,903 Hill Boulevard,41.42,-87.35,$26168,$53350,$128676,685,5,Money Transfer
4,7475332,2010-01-01 00:06:00,848,3915,$46.41,Swipe Transaction,13051,Harwood,5813,Visa,...,Male,166 River Drive,38.86,-76.60,$33529,$68362,$96182,711,2,Drinking Places (Alcoholic Beverages)


In [13]:
valid_ids = fraud_labels_df['id'].unique()

filtered_data = merged_data[merged_data['id'].isin(valid_ids)]

merged_data = pd.merge(filtered_data, fraud_labels_df, on='id', how='left')

In [14]:
merged_data.head()

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,mcc,card_brand,...,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards,data,label
0,7475327,2010-01-01 00:01:00,1556,2972,$-77.00,Swipe Transaction,59935,Beulah,5499,Mastercard,...,594 Mountain View Street,46.80,-100.76,$23679,$48277,$110153,740,4,Miscellaneous Food Stores,No
1,7475328,2010-01-01 00:02:00,561,4575,$14.57,Swipe Transaction,67570,Bettendorf,5311,Mastercard,...,604 Pine Street,40.80,-91.12,$18076,$36853,$112139,834,5,Department Stores,No
2,7475329,2010-01-01 00:02:00,1129,102,$80.00,Swipe Transaction,27092,Vista,4829,Mastercard,...,2379 Forest Lane,33.18,-117.29,$16894,$34449,$36540,686,3,Money Transfer,No
3,7475332,2010-01-01 00:06:00,848,3915,$46.41,Swipe Transaction,13051,Harwood,5813,Visa,...,166 River Drive,38.86,-76.60,$33529,$68362,$96182,711,2,Drinking Places (Alcoholic Beverages),No
4,7475333,2010-01-01 00:07:00,1807,165,$4.81,Swipe Transaction,20519,Bronx,5942,Mastercard,...,14780 Plum Lane,40.84,-73.87,$25537,$52065,$98613,828,5,Book Stores,No


In [15]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8914963 entries, 0 to 8914962
Data columns (total 35 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   id                     int64  
 1   date                   object 
 2   client_id              int64  
 3   card_id                int64  
 4   amount                 object 
 5   use_chip               object 
 6   merchant_id            int64  
 7   merchant_city          object 
 8   mcc                    int64  
 9   card_brand             object 
 10  card_type              object 
 11  card_number            int64  
 12  expires                object 
 13  cvv                    int64  
 14  has_chip               object 
 15  num_cards_issued       int64  
 16  credit_limit           object 
 17  acct_open_date         object 
 18  year_pin_last_changed  int64  
 19  card_on_dark_web       object 
 20  current_age            int64  
 21  retirement_age         int64  
 22  birth_year        

In [16]:
merged_data.isnull().sum()

id                       0
date                     0
client_id                0
card_id                  0
amount                   0
use_chip                 0
merchant_id              0
merchant_city            0
mcc                      0
card_brand               0
card_type                0
card_number              0
expires                  0
cvv                      0
has_chip                 0
num_cards_issued         0
credit_limit             0
acct_open_date           0
year_pin_last_changed    0
card_on_dark_web         0
current_age              0
retirement_age           0
birth_year               0
birth_month              0
gender                   0
address                  0
latitude                 0
longitude                0
per_capita_income        0
yearly_income            0
total_debt               0
credit_score             0
num_credit_cards         0
data                     0
label                    0
dtype: int64

In [17]:
len(merged_data)

8914963

## Pre-Processing

### Removing unnecessary symbols

In [18]:
currency_columns = ['amount', 'per_capita_income', 'yearly_income', 'total_debt']

for col in currency_columns:
    if col in merged_data.columns:
        merged_data[col] = merged_data[col].replace({'\$': '', ',': ''}, regex=True).astype(float)


<>:5: SyntaxWarning: invalid escape sequence '\$'
<>:5: SyntaxWarning: invalid escape sequence '\$'
C:\Users\rishi\AppData\Local\Temp\ipykernel_13640\3886010193.py:5: SyntaxWarning: invalid escape sequence '\$'
  merged_data[col] = merged_data[col].replace({'\$': '', ',': ''}, regex=True).astype(float)


In [19]:
merged_data.head()

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,mcc,card_brand,...,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards,data,label
0,7475327,2010-01-01 00:01:00,1556,2972,-77.00,Swipe Transaction,59935,Beulah,5499,Mastercard,...,594 Mountain View Street,46.80,-100.76,23679.0,48277.0,110153.0,740,4,Miscellaneous Food Stores,No
1,7475328,2010-01-01 00:02:00,561,4575,14.57,Swipe Transaction,67570,Bettendorf,5311,Mastercard,...,604 Pine Street,40.80,-91.12,18076.0,36853.0,112139.0,834,5,Department Stores,No
2,7475329,2010-01-01 00:02:00,1129,102,80.00,Swipe Transaction,27092,Vista,4829,Mastercard,...,2379 Forest Lane,33.18,-117.29,16894.0,34449.0,36540.0,686,3,Money Transfer,No
3,7475332,2010-01-01 00:06:00,848,3915,46.41,Swipe Transaction,13051,Harwood,5813,Visa,...,166 River Drive,38.86,-76.60,33529.0,68362.0,96182.0,711,2,Drinking Places (Alcoholic Beverages),No
4,7475333,2010-01-01 00:07:00,1807,165,4.81,Swipe Transaction,20519,Bronx,5942,Mastercard,...,14780 Plum Lane,40.84,-73.87,25537.0,52065.0,98613.0,828,5,Book Stores,No


### Reformatting the date column

In [5]:
if 'date' in merged_data.columns:
    merged_data['date'] = pd.to_datetime(merged_data['date'])
    merged_data['year'] = merged_data['date'].dt.year
    merged_data['month'] = merged_data['date'].dt.month
    merged_data['day_of_week'] = merged_data['date'].dt.dayofweek
    merged_data['hour'] = merged_data['date'].dt.hour
    merged_data.drop(columns=['date'], inplace=True)

merged_data['mean_transaction_per_client'] = merged_data.groupby('client_id')['amount'].transform('mean')
merged_data['total_transaction_amount'] = merged_data.groupby('client_id')['amount'].transform('sum')

### Converting non-numeric data into numeric data using a Label Encoder

In [6]:
from sklearn.preprocessing import LabelEncoder

categorical_columns = merged_data.select_dtypes(include=['object']).columns

for col in categorical_columns:
    le = LabelEncoder()
    merged_data[col] = le.fit_transform(merged_data[col])

### Scaling down the integer and float data type columns using Standard Scaler for Normalization

In [7]:
from sklearn.preprocessing import StandardScaler

numerical_columns = merged_data.select_dtypes(include=['float64', 'int64']).columns

scaler = StandardScaler()
merged_data[numerical_columns] = scaler.fit_transform(merged_data[numerical_columns])

In [8]:
merged_data.head()

,id,client_id,card_id,amount,use_chip,merchant_id,merchant_city,mcc,card_brand,card_type,...,credit_score,num_credit_cards,data,label,year,month,day_of_week,hour,mean_transaction_per_client,total_transaction_amount
0,-1.723942,0.910066,-0.300334,-1.471293,2,0.472933,817,-0.075496,2,2,...,0.396169,0.101296,64,0,2010,1,4,0,0.043814,-1.003944
1,-1.723941,-0.800510,0.657009,-0.348100,2,0.768677,816,-0.290228,2,0,...,1.824418,0.739173,26,0,2010,1,4,0,-0.879303,-0.715074
2,-1.723941,0.175980,-2.014353,0.454461,2,-0.799251,11332,-0.840766,2,1,...,-0.424314,-0.536581,70,0,2010,1,4,0,0.365762,0.394397
3,-1.723941,-0.307107,0.262844,0.042448,2,-1.343134,4595,0.283153,3,1,...,-0.044460,-1.174457,32,0,2010,1,4,0,1.601069,-0.320728
4,-1.723940,1.341578,-1.976728,-0.467816,2,-1.053859,1226,0.430496,2,2,...,1.733253,0.739173,12,0,2010,1,4,0,0.207155,0.985679


### splitting the dataset into training and testing sets using train_test_split, with the 'label' column as the dependent variable

In [9]:
from sklearn.model_selection import train_test_split

X = merged_data.drop(columns=['label'])
y = merged_data['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Training data: {X_train.shape}, Test data: {X_test.shape}")

Training data: (7131970, 39), Test data: (1782993, 39)


### Used SMOTE to balance the dataset by generating new data points for the minority class

In [10]:
from imblearn.over_sampling import SMOTE
from collections import Counter

smote = SMOTE(random_state=42, k_neighbors=1, sampling_strategy = 0.5)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

print("Original class distribution:", Counter(y_train))
print("Resampled class distribution:", Counter(y_resampled))

Original class distribution: Counter({0: 7121304, 1: 10666})
Resampled class distribution: Counter({0: 7121304, 1: 3560652})


### Built a Random Forest classifier model

In [11]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_resampled, y_resampled)

RandomForestClassifier(random_state=42)

### Used Evaluation metrics to check the model's prediction accuracy on the testing dataset

In [12]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

train_predictions = rf_model.predict(X_train)
train_accuracy = accuracy_score(y_train, train_predictions)
print(f"Training Accuracy: {train_accuracy:.2f}")

Training Accuracy: 1.00


In [13]:
y_pred = rf_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f"Test Accuracy: {accuracy:.2f}")
print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)

Test Accuracy: 1.00
Confusion Matrix:
[[1780286      41]
 [    661    2005]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1780327
           1       0.98      0.75      0.85      2666

    accuracy                           1.00   1782993
   macro avg       0.99      0.88      0.93   1782993
weighted avg       1.00      1.00      1.00   1782993

